# Importing Modules

In [34]:
import pandas as pd
import numpy as np
import imdb

# Reading Data into Data Frames

In [35]:
revenues = pd.read_csv('movies-revenue.csv')
directors = pd.read_csv('movie-director.csv')
voice_actors = pd.read_csv('movie-voice-actors.csv')

# Joining Data into a Single Data Frame

In [36]:
revenues.head(1)

,movie_title,release_date,genre,MPAA_rating,revenue
0,Recess: School's Out,16-Feb-01,Comedy,G,"$54,656,124"


In [37]:
directors.head(1)

,name,director
0,Snow White and the Seven Dwarfs,David Hand


In [38]:
voice_actors.head(1)

,character,voice-actor,movie
0,Abigail Gabble,Monica Evans,The Aristocats


In [39]:
directors.rename(columns={'name': 'movie_title'}, inplace=True)
voice_actors.rename(columns={'movie': 'movie_title'}, inplace=True)

In [40]:
directors.head(1)

,movie_title,director
0,Snow White and the Seven Dwarfs,David Hand


In [41]:
voice_actors.head(1)

,character,voice-actor,movie_title
0,Abigail Gabble,Monica Evans,The Aristocats


In [42]:
revenues_directors = pd.merge(revenues, directors, on="movie_title", how="outer")

In [43]:
revenues_directors.head(1)

,movie_title,release_date,genre,MPAA_rating,revenue,director
0,Recess: School's Out,16-Feb-01,Comedy,G,"$54,656,124",NaN


In [44]:
data = pd.merge(revenues_directors, voice_actors, on="movie_title", how="outer")
data.head(1)

,movie_title,release_date,genre,MPAA_rating,revenue,director,character,voice-actor
0,Recess: School's Out,16-Feb-01,Comedy,G,"$54,656,124",NaN,NaN,NaN


In [45]:
data.isna().sum()

movie_title       0
release_date    353
genre           367
MPAA_rating     458
revenue         353
director        714
character       424
voice-actor     424
dtype: int64

In [46]:
data.shape

(1250, 8)

In [47]:
data = data.dropna(axis=0, subset=['revenue'])

In [48]:
data.isna().sum()

movie_title       0
release_date      0
genre            14
MPAA_rating     105
revenue           0
director        444
character       424
voice-actor     424
dtype: int64

In [49]:
data.shape

(897, 8)

# Parsing Dates and Floats

In [50]:
data['release_date'].dtype

dtype('O')

In [51]:
data['revenue'].dtype

dtype('O')

In [52]:
date_lengths = data['release_date'].str.len()
date_lengths.value_counts()

9    662
8    235
Name: release_date, dtype: int64

In [54]:
data['release_date'].head(1)

0    16-Feb-01
Name: release_date, dtype: object

In [55]:
data['release_date'] = pd.to_datetime(data['release_date'])

In [56]:
data['release_date'].head(1)

0   2001-02-16
Name: release_date, dtype: datetime64[ns]

In [57]:
data['revenue'].head(1)

0    $54,656,124 
Name: revenue, dtype: object

In [58]:
data['revenue'] = data['revenue'].replace("[$,]", "", regex=True).astype(float)

In [59]:
data['revenue'].head(1)

0    54656124.0
Name: revenue, dtype: float64

In [60]:
data.shape

(897, 8)

In [61]:
data.isna().sum()

movie_title       0
release_date      0
genre            14
MPAA_rating     105
revenue           0
director        444
character       424
voice-actor     424
dtype: int64

# Filling NaNs Using ImdbPY Library

## Filling Movies' Genre

In [ ]:
ia = imdb.IMDb()

In [64]:
genre_nans = data[data['genre'].isna()].movie_title
genre_nans

85                             Bound by Honor
130                  The Devil and Max Devlin
138             The Last Flight of Noah’s Ark
234                       My Boyfriend's Back
278    The Many Adventures of Winnie the Pooh
389                           The War at Home
507                  High Heels and Low Lifes
540                             Red Rock West
619                       Frank McKlusky C.I.
625                Herbie Goes to Monte Carlo
703                            The Black Hole
705              A Gun in Betty Lou's Handbag
840                               Passed Away
869                               Father Hood
Name: movie_title, dtype: object

In [69]:
for name in genre_nans:
    search = ia.search_movie(name)
    id = search[0].movieID
    movie = ia.get_movie(id)
    genre = movie['genres'][0]
    data['genre'].loc[data['movie_title'] == name] = genre


C:\Users\-O-O-\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [75]:
data.isna().sum()

movie_title       0
release_date      0
genre             0
MPAA_rating     105
revenue           0
director        444
character       424
voice-actor     424
dtype: int64

## Filling Movies MPAA Rating

In [78]:
rating_nans = data[data['MPAA_rating'].isna()].movie_title
rating_nans.head()

89                         Hello Again
113                         Tough Guys
114    Something Wicked This Way Comes
124                     Never Cry Wolf
130           The Devil and Max Devlin
Name: movie_title, dtype: object

In [84]:
name = "Inception"
search = ia.search_movie(name)
id = search[0].movieID
movie = ia.get_movie(id)

ratingsLen = len(movie.data['certificates'])
ratings = movie.data['certificates']

for i in range(ratingsLen):
    rating = certfificate[i]
    if 'United States' in rating:
        rating = rating.split(":", 1)[1]
        if rating in MPAA_ratings:
            print(certificates)

PG-13


In [89]:
i = 0
#for name in rating_nans:
name = "Hello Again"
search = ia.search_movie(name)
id = search[0].movieID
movie = ia.get_movie(id)
    
ratingsLen = len(movie.data['certificates'])
ratings = movie.data['certificates']

for i in range(ratingsLen):
    rating = certfificate[i]
    if 'United States' in rating:
        rating = rating.split(":", 1)[1]
        if rating in MPAA_ratings:
            data['MPAA_rating'].loc[data['movie_title'] == name] = rating
   

In [90]:
data.isna().sum()

movie_title       0
release_date      0
genre             0
MPAA_rating     105
revenue           0
director        444
character       424
voice-actor     424
dtype: int64